<a href="https://colab.research.google.com/github/evaie/evaie/blob/main/process_inscriptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get authentication for reading the google sheet

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

Read the google sheet and transform it in a dataframe properly


In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1SCruyMO_mncyvIF8kA9HEN2cTabM8ge3Mj2Ia_yG8Uk/edit#gid=464665913').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
df_inscriptions = pd.DataFrame.from_records(rows,columns=rows[0])
df_inscriptions = df_inscriptions.drop(0).reindex()

detect the malformed naissance columsns and change the date formate

In [ ]:
df_inscriptions.dropna()
df_inscriptions['naissance'].count()
df_ddl_1 = df_inscriptions[df_inscriptions['naissance']=='0000-00-00']
df_ddl_1

,saison,statut,date_statut,civil,nom,prenom,prise_licence,email,tel,adresse,cp,ville,pays,nationalite,naissance,profession,entreprise,parent1_civil,parent1_nom,parent1_prenom,parent1_email,parent1_tel,parent2_civil,parent2_nom,parent2_prenom,parent2_email,parent2_tel,contact_civil,contact_nom,contact_prenom,contact_email,contact_tel,docteur_nom,docteur_date,handicap,reglement,documentation,benevolat,nager,assurance,type,licence,categorie,garage,optioncomp,montant,reduction,motifreduction,netapayer,paiement_helloasso,virement,cheque,nbcheques,chequesvacances,couponsport,pass92,caution_nom,caution,especes,autre_details,autre,montant_total
23,2021-2022,Validé,2021-08-31 11:49:45,Monsieur,EXBRAYAT,JEAN-LOUIS,oui,jexbrayat@icloud.com,06 69 05 22 63?,71 rue Victor Hugo,92400,Courbevoie,FR,FR,0000-00-00,retrairé,,,,,,,,,,,,Madame,EXBRAYAT,ELYANE,elexbrayat@gmail.com,06 64 26 24 16?,Fanny Exbrayat,2020-09-04,non,on,on,on,on,on,renouvellement,432020,Séniors + Loisirs + CE,non,non,460.00,0.00,,460.00,0.00,0.00,460.00,1,0.00,0.00,0.00,,0.00,0.00,,0.00,460.00


In [ ]:
df_ddl_2 =df_inscriptions[df_inscriptions['naissance']=='']
df_ddl_2

,saison,statut,date_statut,civil,nom,prenom,prise_licence,email,tel,adresse,cp,ville,pays,nationalite,naissance,profession,entreprise,parent1_civil,parent1_nom,parent1_prenom,parent1_email,parent1_tel,parent2_civil,parent2_nom,parent2_prenom,parent2_email,parent2_tel,contact_civil,contact_nom,contact_prenom,contact_email,contact_tel,docteur_nom,docteur_date,handicap,reglement,documentation,benevolat,nager,assurance,type,licence,categorie,garage,optioncomp,montant,reduction,motifreduction,netapayer,paiement_helloasso,virement,cheque,nbcheques,chequesvacances,couponsport,pass92,caution_nom,caution,especes,autre_details,autre,montant_total
80,,,,,,,34,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [ ]:
df_inscriptions = df_inscriptions[df_inscriptions['naissance']!='0000-00-00']
df_inscriptions = df_inscriptions[df_inscriptions['naissance']!='']

In [ ]:
from datetime import datetime
df_inscriptions['naissance']= df_inscriptions[df_inscriptions['naissance']!='0000-00-00']['naissance'].\
apply(lambda x : datetime.strptime(x, '%Y-%m-%d').strftime('%d/%m/%Y'))

In [ ]:
df_inscriptions['sexe'] = df_inscriptions['civil'].apply(lambda x : 'H' if x == 'Monsieur' else 'F')

In [ ]:
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(list(df_inscriptions.filter(regex='^parent')))]
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(list(df_inscriptions.filter(regex='^contact')))]
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(list(df_inscriptions.filter(regex='[a-z]*cheque[a-z]*')))]
df_inscriptions = df_inscriptions[df_inscriptions.columns.drop(['documentation', 'benevolat', 'nager', 'assurance',\
                                                                'caution_nom', 'caution', 'especes', 'autre_details',\
                                                                'montant_total','saison','statut','date_statut','profession',\
                                                                'entreprise','reglement','garage','montant','reduction',\
                                                                'motifreduction','netapayer', 'paiement_helloasso','virement',\
                                                                'couponsport','pass92','optioncomp','autre'])]

In [ ]:
from datetime import date,timedelta
yesterday = date.today() - timedelta(days=1)
yester_str = yesterday.strftime('%Y-%m-%d')

In [ ]:
df_inscriptions['docteur_date'] = df_inscriptions['docteur_date'].apply(lambda x : yester_str if x =='0000-00-00' else x)
df_inscriptions['docteur_date']

1     2021-06-18
2     2021-09-09
3     2020-09-10
4     2021-09-18
5     2021-09-18
         ...    
75    2021-09-18
76    2002-01-25
77    2021-07-07
78    2020-08-04
79    2020-10-09
Name: docteur_date, Length: 78, dtype: object

In [ ]:
from datetime import datetime
df_inscriptions['docteur_date']= df_inscriptions['docteur_date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d').strftime('%d/%m/%Y'))

In [ ]:
df_inscriptions['docteur_nom'] = df_inscriptions['docteur_nom'].apply(lambda x : str.upper(x))
df_inscriptions['ville'] = df_inscriptions['ville'].apply(lambda x : str.upper(x))

In [ ]:
try :
  inscription_reprocessed = worksheet.spreadsheet.worksheet('re-processed')
  inscription_reprocessed.clear()
except:
  inscription_reprocessed = worksheet.spreadsheet.add_worksheet(title="re-processed", rows="350", cols="20")

In [ ]:
from gspread_dataframe import set_with_dataframe
set_with_dataframe(inscription_reprocessed, df_inscriptions)